In [1]:
from sklearn.model_selection import StratifiedKFold
from scipy import stats
import numpy as np
from sklearn import linear_model, svm
from sklearn.metrics import roc_curve, auc,f1_score
import matplotlib.pyplot as plt
import os
import pandas as pd
from sklearn.feature_selection import SelectFromModel

print(os.getcwd())

/Users/yunwang/Documents/research/Jiook_AD/papers/idp_jiook/src


In [2]:
df = pd.read_csv('../braindata/data_2_conn_select_10000.csv',header=0)


In [3]:
data=np.array(df)

In [4]:
print(data.shape)


(211, 33709)


In [5]:
idx_IN_columns = np.append(np.array([3,4]),np.array(range(11,data.shape[1])))
features=data[:,idx_IN_columns]

In [6]:
features = stats.zscore(features)
print(features.shape)

(211, 33700)


In [7]:
y=data[:,6]
print(y.shape)

(211,)


In [8]:
ind_num=np.isnan(y)
print(ind_num.shape)

(211,)


In [9]:
data

array([[  1.00043620e+07,   3.00000000e+00,   4.00000000e+00, ...,
          1.21910000e+02,   1.64950000e+02,   1.34640000e+02],
       [  1.00054260e+07,   1.00000000e+00,   1.00000000e+00, ...,
          1.35780000e+02,   1.63970000e+02,   1.65160000e+02],
       [  1.00110000e+07,   2.00000000e+00,   3.00000000e+00, ...,
          1.07550000e+02,   1.48830000e+02,   1.64640000e+02],
       ..., 
       [  1.14168270e+07,   2.00000000e+00,   3.00000000e+00, ...,
          7.31560000e+01,   6.71800000e+01,   0.00000000e+00],
       [  1.14212080e+07,   1.00000000e+00,   2.00000000e+00, ...,
          1.40280000e+02,   1.72720000e+02,   1.70530000e+02],
       [  1.14337630e+07,   3.00000000e+00,   4.00000000e+00, ...,
          5.40260000e+01,   7.09370000e+01,   2.22450000e+02]])

In [10]:
range(10)

range(0, 10)

In [11]:
#features = np.loadtxt(open("../braindata/Y_rho_03.csv", "rb"), delimiter=",", skiprows=0)
#features = np.loadtxt(open("../braindata/data_1_mor_select_100.csv", "rb"), delimiter=",", skiprows=1)
#df = pd.read_csv('../braindata/data_1_mor_select_100.csv',header=0)
df = pd.read_csv('../braindata/data_2_conn_select_10000.csv',header=0)
data=np.array(df)
print(data.shape)
idx_IN_columns = np.append(np.array([3,4]),np.array(range(11,data.shape[1])))
features=data[:,idx_IN_columns]
#features=data[:,11:data.shape[1]]
#features = features.transpose()
features = stats.zscore(features)
print(features.shape)

#y = np.loadtxt(open("../braindata/dx2.csv", "rb"), delimiter=",", skiprows=1,usecols=range(1,209))
#y = np.loadtxt(open("../braindata/dx3.csv", "rb"), delimiter=",", skiprows=1,usecols=range(1,209))
y=data[:,6]
print(y.shape)
#5: ad-smi / 6:mci-smi / 7:adonly-smi / 8:ad-mci / 9:adonly-mci / 10:adonly - adwithsmallvv

ind_num=np.isnan(y)
print(ind_num.shape)
y_no_nan = y[~ind_num]
features_no_nan = features[~ind_num,:]

print(y.shape)

y=y_no_nan
features=features_no_nan

(211, 33709)
(211, 33700)
(211,)
(211,)
(211,)


In [12]:
features


array([[ 0.6672372 , -1.213215  ,  2.85125306, ...,  0.01223059,
         0.81304876,  0.24098655],
       [ 0.6672372 ,  0.46618383, -0.25162921, ..., -0.32029688,
         0.34183172,  0.78620908],
       [-1.4987174 , -0.85334382, -0.48147234, ...,  0.69442133,
         0.16673307, -2.20597218],
       ..., 
       [-1.4987174 ,  0.94601207, -0.71131547, ..., -2.81077939,
        -4.00874104, -2.20597218],
       [ 0.6672372 , -2.17287147, -0.71131547, ..., -1.11674184,
        -2.04494693, -2.20597218],
       [ 0.6672372 , -1.69304323,  3.54078245, ..., -1.55972585,
        -1.93512295,  1.8368529 ]])

In [13]:
y

array([ 0.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  1.,
        0.,  1.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  0.,  0.,  1.,  1.,
        1.,  1.,  1.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,
        0.,  0.,  1.,  1.,  0.,  0.,  1.,  1.,  0.,  1.,  1.,  0.,  1.,
        0.,  1.,  1.,  1.,  0.,  0.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,
        1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  1.,  1.,
        1.,  0.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        0.,  1.,  0.,  1.,  1.,  0.,  1.,  0.,  1.,  0.])

In [19]:
lr_all_accs = []
lr_all_scores = []
lr_f1s = []
svm_all_accs = []
svm_all_scores = []
svm_f1s = []
base_labels = []


for runs in range(10):
    lr_accuracies = []
    lr_scores = []
    svm_accuracies = []
    svm_scores = []
    strat_labels = []

    rbf = svm.SVC(C=10,kernel='linear',gamma=0.01)
    logistic = linear_model.LogisticRegression(C=1e5)

    skf = StratifiedKFold(n_splits=10)
    for train_index, test_index in skf.split(features, y):
        train_data, test_data = features[train_index], features[test_index]
        train_labels, test_labels = y[train_index], y[test_index]
        
        strat_labels = np.append(strat_labels, test_labels)

        log = logistic.fit(train_data, train_labels)
        log_prob = log.decision_function(test_data)
        log_acc = log.score(test_data, test_labels)
        
        #f1 calculation
        y_pred = logistic.predict(test_data)
        log_f1 = f1_score(test_labels, y_pred)

        lr_accuracies = np.append(lr_accuracies, log_acc)
        lr_scores = np.append(lr_scores, log_prob)
        lr_f1s = np.append(lr_f1s, log_f1)
        #print('Logistic Regression Accuracy: %f' % log_acc)

        rbf = rbf.fit(train_data, train_labels)
        svm_acc = rbf.score(test_data, test_labels)
        svm_prob = rbf.decision_function(test_data)
        
        #f1 calculation
        y_pred = rbf.predict(test_data)
        svm_f1 = f1_score(test_labels, y_pred)
        #print('SVM Accuracy: %f' % svm_acc)
        svm_accuracies = np.append(svm_accuracies, svm_acc)
        svm_scores = np.append(svm_scores, svm_prob)
        svm_f1s = np.append(svm_f1s, svm_f1)
   
        base_labels = np.append(base_labels, strat_labels)
        lr_all_scores = np.append(lr_all_scores, lr_scores)
        lr_fold_avg = np.mean(lr_accuracies)
        lr_all_accs = np.append(lr_all_accs, lr_fold_avg)
        svm_all_scores = np.append(svm_all_scores, svm_scores)
        svm_fold_avg = np.mean(svm_accuracies)
        svm_all_accs = np.append(svm_all_accs, svm_fold_avg)

/Users/yunwang/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [17]:
svm_all_accs

array([ 1.        ,  1.        ,  0.96969697,  0.93181818,  0.92545455,
        0.92121212,  0.93246753,  0.89924242,  0.84870932,  0.78606061,
        1.        ,  1.        ,  0.96969697,  0.93181818,  0.92545455,
        0.92121212,  0.93246753,  0.89924242,  0.84870932,  0.78606061,
        1.        ,  1.        ,  0.96969697,  0.93181818,  0.92545455,
        0.92121212,  0.93246753,  0.89924242,  0.84870932,  0.78606061,
        1.        ,  1.        ,  0.96969697,  0.93181818,  0.92545455,
        0.92121212,  0.93246753,  0.89924242,  0.84870932,  0.78606061,
        1.        ,  1.        ,  0.96969697,  0.93181818,  0.92545455,
        0.92121212,  0.93246753,  0.89924242,  0.84870932,  0.78606061,
        1.        ,  1.        ,  0.96969697,  0.93181818,  0.92545455,
        0.92121212,  0.93246753,  0.89924242,  0.84870932,  0.78606061,
        1.        ,  1.        ,  0.96969697,  0.93181818,  0.92545455,
        0.92121212,  0.93246753,  0.89924242,  0.84870932,  0.78

In [18]:
print(svm_all_accs.shape)

(100,)


In [21]:
import numpy as np
import pandas as pd
from pprint import pprint
from os import listdir
from os.path import isfile, join
from sklearn.model_selection import StratifiedKFold
from scipy import stats
import numpy as np
from sklearn import linear_model, svm

df = pd.read_csv('../braindata/data_1_mor.csv',header=0, index_col=0)
df['sum'] = df.sum(axis=1, numeric_only=True)

In [22]:
df


,dx.1ad.2mci.3smi,dx2.1adonly.2adwithsmallvv.3mci.4.smi,SEX,AGE,dx_ad_smi,dx_mci_smi,dx_adonly_smi,dx_ad_mci,dx_adonly_mci,dx_adonly_adwithsmallvv,...,SupraTentorialVolNotVent,SupraTentorialVolNotVentVox,MaskVol,BrainSegVol.to.eTIV,MaskVol.to.eTIV,lhSurfaceHoles,rhSurfaceHoles,SurfaceHoles,EstimatedTotalIntraCranialVol,sum
subject,,,,,,,,,,,,,,,,,,,,,
10004362,3,4,2,66,0.0,0.0,0.0,NaN,NaN,NaN,...,9.540080e+05,951754,1536156,0.733417,1.019869,94,101,195,1506228.423,1.875730e+07
10005426,1,1,2,90,1.0,NaN,1.0,1.0,1.0,1.0,...,7.752900e+05,773532,1481686,0.603025,0.907955,95,86,181,1631893.520,1.700531e+07
10011000,2,3,2,80,NaN,1.0,NaN,0.0,0.0,NaN,...,7.893957e+05,787964,1376469,0.661764,0.980232,119,117,236,1404228.400,1.617731e+07
10014793,1,1,2,76,1.0,NaN,1.0,1.0,1.0,1.0,...,8.664247e+05,864554,1640724,0.696777,1.057352,85,100,185,1551729.752,1.813539e+07
10015858,2,3,1,69,NaN,1.0,NaN,0.0,0.0,NaN,...,1.072179e+06,1069822,1676706,0.737229,0.988962,35,32,67,1695420.197,2.083721e+07
10016158,3,4,2,73,0.0,0.0,0.0,NaN,NaN,NaN,...,9.960037e+05,994177,1614961,0.718843,0.991091,42,39,81,1629478.772,1.968096e+07
10021984,2,3,2,75,NaN,1.0,NaN,0.0,0.0,NaN,...,9.033262e+05,899274,1513699,0.704459,1.028357,93,122,215,1471958.711,1.793365e+07
10029184,1,1,1,93,1.0,NaN,1.0,1.0,1.0,1.0,...,8.416888e+05,839723,1579143,0.654204,1.014423,80,82,162,1556691.135,1.766222e+07
10031650,2,3,2,73,NaN,1.0,NaN,0.0,0.0,NaN,...,8.733584e+05,871936,1450864,0.705222,1.013554,67,63,130,1431461.963,1.729933e+07


In [23]:
print(df.shape)

(208, 959)
